In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 160)
print pd.__version__

0.17.1


In [2]:
df = pd.read_csv("./attributes.csv", dtype={'value': object})
df = df[~pd.isnull(df.name)]
df[df.product_uid == 100002]

,product_uid,name,value
15,100002,Application Method,"Brush,Roller,Spray"
16,100002,Assembled Depth (in.),6.63 in
17,100002,Assembled Height (in.),7.76 in
18,100002,Assembled Width (in.),6.63 in
19,100002,Bullet01,"Revives wood and composite decks, railings, porches and boat docks, also great for concrete pool decks, patios and sidewalks"
20,100002,Bullet02,100% acrylic solid color coating
21,100002,Bullet03,Resists cracking and peeling and conceals splinters and cracks up to 1/4 in.
22,100002,Bullet04,"Provides a durable, mildew resistant finish"
23,100002,Bullet05,Covers up to 75 sq. ft. in 2 coats per gallon
24,100002,Bullet06,"Creates a textured, slip-resistant finish"


In [3]:
print df.describe()
print df.columns.values
print df.dtypes

          product_uid
count  2044648.000000
mean    170495.654144
std      36374.522113
min     100001.000000
25%     140463.750000
50%     177406.000000
75%     200558.000000
max     224428.000000
['product_uid' 'name' 'value']
product_uid    float64
name            object
value           object
dtype: object


In [4]:
# Many of the values are true/false/yes/no properties that we should transform into the name of the value. For example,
# one yes/no value is for the attribute "Zippered Top", let's change the value to "ZipperedTop" or "NonZipperedTop"
binary_attrs = df[(df.value.str.lower() == 'yes') | (df.value.str.lower() == 'no')].name.unique()
print np.sort(binary_attrs)

idx = df.name.isin(binary_attrs) & (df.value.str.lower() == 'yes')
df.loc[idx, 'value'] = df[idx].name.map(lambda x: str(x).replace(' ', ''))

idx = df.name.isin(binary_attrs) & (df.value.str.lower() == 'no')
df.loc[idx, 'value'] = df[idx].name.map(lambda x: 'Non' + str(x).replace(' ', ''))

df[df.name.isin(binary_attrs)].head()

['15 Gauge Finish Nailer Included' '16 Gauge Finish Nailer Included'
 '18 Gauge Finish Stapler Included' ..., 'Yaw Adjustment' 'Zippered Top'
 'Zone-specific Sounds']


,product_uid,name,value
32,100002,Concrete Use,ConcreteUse
35,100002,Deck Use,DeckUse
38,100002,Mildew Resistant,MildewResistant
44,100002,Sealer,NonSealer
46,100002,Tintable,NonTintable


In [5]:
# If the name begins with 'number of', put the object in the value. Ex) "2" -> "2 Panels"

# Standardize the identifying phrase for measurement attributes
df.loc[df.name.str.lower().str.startswith('# of'), 'name'] = df.name.map(lambda x: str(x).replace('# of', 'Number of'))

# Find all the unique measurement attributes
count_attrs = df[df.name.str.lower().str.startswith('number of')].name.unique()
print np.sort(count_attrs)[:10]

# Prepend the attribute name (without 'Number of') to the value
idx = df.name.isin(count_attrs)
df.loc[idx, 'value'] = df[idx].value + ' ' + df[idx].name.map(lambda x: str(x).replace('Number of', ''))
df[idx].head()

# We could make further modifications to change values of "0" or "None" to "No" so it is closer to what a search term may
# be. Ex) "0 Bulbs Required" -> "No Bulbs Required"

['Number of Activities' 'Number of Adjustable Correction Levels'
 'Number of Adjusting Holes' 'Number of Anchor Points'
 'Number of Attachments' 'Number of BNC' 'Number of Balanced Audio Inputs'
 'Number of Balanced Audio Outputs' 'Number of Batteries Included'
 'Number of Batteries Required']


,product_uid,name,value
10,100001,Number of Pieces,1 Pieces
95,100004,Number of Panels,4 Panels
123,100005,Number of Faucet Handles,Single Handle Faucet Handles
124,100005,Number of showerheads,1 showerheads
125,100005,Number of Spray Settings,1 Spray Settings


In [6]:
# Many of the values are dimensional properties (len, width, etc.) that we can prefix with the measure type
# Ex) "8" -> "A/C Coverage Area (sq. ft.) 8"

# We'll want to exclude the measurement attributes that start with "Number of"
numof = df.name.str.lower().str.startswith('number of')

# Build a list of all the other measurement attributes
measure_attrs = []
measure_attrs.extend(df[~numof & df.name.str.contains("depth", case=False)].name.unique())
measure_attrs.extend(df[~numof & df.name.str.contains("height", case=False)].name.unique())
measure_attrs.extend(df[~numof & df.name.str.contains("length", case=False)].name.unique())
measure_attrs.extend(df[~numof & df.name.str.contains("width", case=False)].name.unique())
measure_attrs.extend(df[~numof & df.name.str.contains("thickness", case=False)].name.unique())
measure_attrs.extend(df[~numof & df.name.str.contains("diameter", case=False)].name.unique())
measure_attrs.extend(df[~numof & df.name.str.contains("temperature", case=False)].name.unique())
measure_attrs.extend(df[~numof & df.name.str.contains("area \(", case=False)].name.unique())
measure_attrs.extend(df[~numof & df.name.str.contains("area covered", case=False)].name.unique())
measure_attrs.extend(df[~numof & df.name.str.contains("size", case=False)].name.unique())
measure_attrs.extend(df[~numof & df.name.str.contains("opening \(", case=False)].name.unique())
measure_attrs = np.array(np.sort(list(set(measure_attrs))))
print measure_attrs

# Prepend the name to the value
idx = df.name.isin(measure_attrs)
df.loc[idx, 'value'] = df[idx].name + ' ' + df[idx].value
df[idx].head()

['A/C Coverage Area (sq. ft.)' 'Actual Blind/Shade Width (In.)'
 'Actual Color Temperature (K)' ..., 'Wreath Diameter (In.)' 'Wrench Size'
 'Wrench length (in.)']


,product_uid,name,value
11,100001,Product Depth (in.),Product Depth (in.) 1.5
12,100001,Product Height (in.),Product Height (in.) 3
14,100001,Product Width (in.),Product Width (in.) 3
16,100002,Assembled Depth (in.),Assembled Depth (in.) 6.63 in
17,100002,Assembled Height (in.),Assembled Height (in.) 7.76 in


In [7]:
# Combine all the attributes from the same product into a single value
df['attributes'] = df.groupby('product_uid')['value'].transform(lambda x: ' '.join(map(str, x)))
df = df[['product_uid', 'attributes']].drop_duplicates()
df.head()

,product_uid,attributes
0,100001,Versatile connector for various 90° connections and home repair projects Stronger than angled nailing or screw fastening alone Help ensure joints are consis...
15,100002,"Brush,Roller,Spray Assembled Depth (in.) 6.63 in Assembled Height (in.) 7.76 in Assembled Width (in.) 6.63 in Revives wood and composite decks, railings, po..."
50,100003,"Built-inflange Slightly narrower for tighter spaces Designed with an 18 in. apron Durable high-gloss finish provides a smooth, shiny surface that is easy to..."
82,100004,8.56 Positive power tolerance (0 to +5-Watt) Anti-reflective and anti-soiling surface reduces power loss from dirt and dust Outstanding performance in low-l...
107,100005,"Combo Tub and Shower NonBuilt-inWaterFilter Includes the trim kit only, the rough-in kit (R10000-UNBX) is sold separately Includes the handle Maintains a ba..."


In [8]:
### After further investigation, it turns out that the "Descriptions" document contains very redundant information.
### It's only a (error-filled) concatentation of all the bullets in the attributes file, so we'll just ignore it

In [9]:
#
# Before we turn the documents into feature vectors, there's a little bit more cleanup for us to do...
#

# Convert to all lowercase
df.document = df.attributes.map(lambda x: x.lower())

# Remove all dashes from inbetween alphas
import re
regex = r'([a-zA-Z])\-([a-zA-Z])'
df.document = df.document.map(lambda x: re.sub(regex, '\1\2', x))

# remove parens
df.document = df.document.str.replace('(', '')
df.document = df.document.str.replace(')', '')
df.document

# remove degree character
df.document = df.document.str.replace('°', '')

In [10]:
###########################################################################################################
#
# Now that we've massaged the data into a format where each product id has it's own 'document' we're ready 
# to run some TF-IDF code to figure out which words are most important within each document
#
###########################################################################################################
# Control the number of unique words and n-grams we want to include as features. The more, the slower
maxfeatures = 10000

# Minimum document frequency -> any term must appear in at at least this many documents (or % for float values).
# 1 = use all terms in all documents. .005 = use terms that appear in at least .5% of documents
mindocfreq = 1

# Maximum document frequency -> any term must appear in fewer than this many documents (or % for float values)
# .5 means any term occuring in more than 50% of documents is ignored
maxdocfreq = .5

# N-gram range specifies the largest number of words to group as a phrase. 
#    1-gram: 'Is', 'this', 'your', 'homework', 'Larry'
#    2-gram: 'Is this', 'this your', 'your homework', 'homework Larry'
#    3-gram: 'Is this your', 'this your homework', 'your homework Larry'
# We'll only use 1- and 2-grams (aka bi-grams) for this example
ngramrange = (1,2)

In [11]:
# We'll also perform stemming where we attempt to truncate the different endings of related words to capture
# the meaning rather than the spelling. Ex) "surf", "surfs", "surfer", "surfing" -> "surf"
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
df['stemmed'] = df.document.map(lambda x: ' '.join([stemmer.stem(y) for y in x.decode('utf-8').split(' ')]))
df.stemmed.head()

0      versatil connector for various 90 connect and home repair project stronger than angl nail or screw fasten alon help ensur joint are consist straight and str...
15     brush,roller,spray assembl depth in. 6.63 in assembl height in. 7.76 in assembl width in. 6.63 in reviv wood and composit decks, railings, porch and boat do...
50     builnflang slight narrow for tighter space design with an 18 in. apron durabl higloss finish provid a smooth, shini surfac that is easi to clean conform...
82     8.56 posit power toler 0 to +5-watt anteflect and antoil surfac reduc power loss from dirt and dust outstand perform in loight irradi environ certifi ...
107    combo tub and shower nonbuilnwaterfilt includ the trim kit only, the rougn kit r10000-unbx is sold separ includ the handl maintain a balanc pressur of h...
Name: stemmed, dtype: object

In [12]:
# There are two ways to go about this. First, we could use the CountVectorizer to create get the total counts
# of each of the most frequent words, then apply the TfidfTransformer to get the TF-IDF weights. Alternately, we
# could use the TfidfVectorizer to go straight to the weights. Let's do both!
#
# Starting with the CountVectorizer/TfidfTransformer approach...
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cvec = CountVectorizer(min_df=mindocfreq, max_df=maxdocfreq, stop_words='english', ngram_range=ngramrange)
cvec

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [13]:
# Calculate all the n-grams found in all documents
from itertools import islice
cvec.fit(df.stemmed)
list(islice(cvec.vocabulary_.items(), 20))

[(u'168 piec', 27898),
 (u'sand steam', 684841),
 (u'ladder rais', 468211),
 (u'element roug', 322272),
 (u'63 243', 87159),
 (u'includ doubl', 437880),
 (u'spiders', 730976),
 (u'120 fahrenheit', 15372),
 (u'300 escap', 53790),
 (u'nonhammerloop', 551062),
 (u'unhand latch', 805343),
 (u'stress usb', 753406),
 (u'durabl lustrous', 308157),
 (u'brown primary', 187979),
 (u'turntabl upper', 799940),
 (u'ad kit', 118374),
 (u'cluster accent', 226347),
 (u'stress use', 753407),
 (u'mark make', 509375),
 (u'protect virtual', 630679)]

In [14]:
# Check how many total n-grams we have
len(cvec.vocabulary_)

860681

In [35]:
# All documents have somewhere around 860 thousand different ngrams, this is too many. Let's try again 
# with a different min_df that requires a phrase to occur in at least 2 documents
cvec = CountVectorizer(min_df=.0025, max_df=.1, stop_words='english', ngram_range=(1,2))
cvec.fit(df.stemmed)
list(islice(cvec.vocabulary_.items(), 20))

[(u'readi instal', 4127),
 (u'fir', 2062),
 (u'fip', 2061),
 (u'fix', 2076),
 (u'set hand', 4463),
 (u'flush graviti', 2114),
 (u'chain', 1029),
 (u'chair', 1030),
 (u'exact', 1890),
 (u'ground', 2320),
 (u'residenti coverag', 4210),
 (u'bore hole', 844),
 (u'apron', 640),
 (u'conceal shelv', 1208),
 (u'eas instal', 1732),
 (u'effici yea', 1777),
 (u'pivot', 3839),
 (u'cee rate', 1000),
 (u'nort', 3568),
 (u'gang product', 2213)]

In [36]:
len(cvec.vocabulary_)

5616

In [37]:
# Ok, that's great. Now we'll do the transformation to a 'bag of words' representation
cvec_counts = cvec.transform(df.stemmed)
print 'sparse matrix shape:', cvec_counts.shape
print 'nonzero count:', cvec_counts.nnz
print 'sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1]))

sparse matrix shape: (86263, 5616)
nonzero count: 4910003
sparsity: 1.01%


In [38]:
# Let's use the result to find the top terms by total number of occurences
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(20)

,occurrences,term
4942,15518,temperatur
1264,14703,control
130,14155,18
2095,14100,floor
3643,14067,open
917,13743,bulb
177,13624,24
4999,13537,tile
1160,12618,commerci
2240,12551,glass


In [21]:
# From the vectorized count, we can use the TfidfTransformer to calculate the tf-idf weights
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

<86263x10000 sparse matrix of type '<type 'numpy.float64'>'
	with 6582849 stored elements in Compressed Sparse Row format>

In [22]:
# Let's examine the top 20 ngrams by average tf-idf weight
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
1154,assembled,0.030696
5181,length,0.020918
8546,steel,0.018546
3818,ft,0.018040
9671,white,0.017818
9247,use,0.017089
344,25,0.017088
8161,size,0.017044
2871,door,0.016601
643,75,0.016120


In [23]:
################################################################################################################
#
# For fun, let's do this again using the TfidfVectorizer approach instead of CountVectorizer+TfidfTransformer
#
from sklearn.feature_extraction.text import TfidfVectorizer

tvec = TfidfVectorizer(min_df=2, max_df=maxdocfreq, stop_words='english', ngram_range=ngramrange,\
                       max_features=10000)

tvec_weights = tvec.fit_transform(df.stemmed.dropna())
tvec_weights

<86263x10000 sparse matrix of type '<type 'numpy.float64'>'
	with 6582849 stored elements in Compressed Sparse Row format>

In [24]:
# Let's examine the top 20 ngrams by average tf-idf weight, and see how they match up with the previous results
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})

weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
1154,assembled,0.030696
5181,length,0.020918
8546,steel,0.018546
3818,ft,0.018040
9671,white,0.017818
9247,use,0.017089
344,25,0.017088
8161,size,0.017044
2871,door,0.016601
643,75,0.016120
